In [1]:
import pandas as pd

### 查看pandasJSON
JSON是存储和交换文本信息的语法，类似XML
[</br>
   {</br>
   "id": "A001",</br>
   "name": "菜鸟教程",</br>
   "url": "www.runoob.com",</br>
   "likes": 61</br>
   },</br>
   {</br>
   "id": "A002",</br>
   "name": "Google",</br>
   "url": "www.google.com",</br>
   "likes": 124</br>
   },</br>
   {</br>
   "id": "A003",</br>
   "name": "淘宝",</br>
   "url": "www.taobao.com",</br>
   "likes": 45</br>
   }</br>
]

In [2]:
df = pd.read_json('sites.json')
print(df.to_string())

     id    name             url  likes
0  A001    菜鸟教程  www.runoob.com     61
1  A002  Google  www.google.com    124
2  A003      淘宝  www.taobao.com     45


to_string()用于返回DataFrame类型的数据，也可以用于直接处理JSON字符串</br>

In [3]:
data = [
    {
    "id": "A001",
    "name": "菜鸟教程",
    "url": "www.runoob.com",
    "likes": 61
    },
    {
    "id": "A002",
    "name": "Google",
    "url": "www.google.com",
    "likes": 124
    },
    {
    "id": "A003",
    "name": "淘宝",
    "url": "www.taobao.com",
    "likes": 45
    }
]
df = pd.DataFrame(data)
print(df)

     id    name             url  likes
0  A001    菜鸟教程  www.runoob.com     61
1  A002  Google  www.google.com    124
2  A003      淘宝  www.taobao.com     45


JSON对象和Python有相同的格式，所以我们可以将Python字典转换为DataFrame数据

In [4]:
s = {
    "col1":{"row1":1, "row2":2, "row3":3},
    "col2":{"row1":"x", "row2":"y", "row3":"z"}
}
df = pd.DataFrame(s)
print(df)

      col1 col2
row1     1    x
row2     2    y
row3     3    z


从URL中读取JSON数据

In [5]:
URL = 'https://static.runoob.com/download/sites.json'
df = pd.read_json(URL)
print(df)

     id    name             url  likes
0  A001    菜鸟教程  www.runoob.com     61
1  A002  Google  www.google.com    124
2  A003      淘宝  www.taobao.com     45


### 内嵌的JSON数据

In [6]:
df = pd.read_json('nested_list.json')
print(df)

          school_name   class  \
0  ABC primary school  Year 1   
1  ABC primary school  Year 1   
2  ABC primary school  Year 1   

                                            students  
0  {'id': 'A001', 'name': 'Tom', 'math': 60, 'phy...  
1  {'id': 'A002', 'name': 'James', 'math': 89, 'p...  
2  {'id': 'A003', 'name': 'Jenny', 'math': 79, 'p...  


用json_normalize()方法将内嵌的数据完整解析出来

In [7]:
import json
with open('nested_list.json', 'r') as f:
    data = json.loads(f.read())

#展开数据
df_nested_list = pd.json_normalize(data, record_path=['students'])
print(df_nested_list)

     id   name  math  physics  chemistry
0  A001    Tom    60       66         61
1  A002  James    89       76         51
2  A003  Jenny    79       90         78


其他元素也可以用meta在显示

In [8]:
df_nested_list = pd.json_normalize(data, 
                                   record_path=['students'], 
                                   meta=['school_name', 'class'])
print(df_nested_list)

     id   name  math  physics  chemistry         school_name   class
0  A001    Tom    60       66         61  ABC primary school  Year 1
1  A002  James    89       76         51  ABC primary school  Year 1
2  A003  Jenny    79       90         78  ABC primary school  Year 1


嵌套了列表和字典的json

In [9]:
with open('nested_mix.json', 'r') as f:
    data = json.loads(f.read())

df = pd.json_normalize(
    data,
    record_path=['students'],
    meta=[
        'class',
        ['info', 'president'],
        ['info', 'contacts', 'tel']
    ]
)

print(df)

     id   name  math  physics  chemistry   class info.president  \
0  A001    Tom    60       66         61  Year 1    John Kasich   
1  A002  James    89       76         51  Year 1    John Kasich   
2  A003  Jenny    79       90         78  Year 1    John Kasich   

  info.contacts.tel  
0         123456789  
1         123456789  
2         123456789  


只读内嵌的math字段

In [11]:
from glom import glom
df = pd.read_json('nested_deep.json')
data = df['students'].apply(lambda row: glom(row, 'grade.math'))
print(data)

0    60
1    89
2    79
Name: students, dtype: int64
